# Import Packages

In [1]:
import pandas as pd
import numpy as np
import glob

# Bring in the data

In [2]:
fileList= glob.glob("../data/*.*")
fileList

['../data\\RegularSeasonCompactResults.csv',
 '../data\\RegularSeasonDetailedResults.csv',
 '../data\\sample_submission.csv',
 '../data\\Seasons.csv',
 '../data\\Teams.csv',
 '../data\\TourneyCompactResults.csv',
 '../data\\TourneyDetailedResults.csv',
 '../data\\TourneySeeds.csv',
 '../data\\TourneySlots.csv']

In [3]:
regSeasonComp = pd.read_csv(fileList[0], verbose=False)
regSeasonDet = pd.read_csv(fileList[1], verbose=False)
sampleSub = pd.read_csv(fileList[2], verbose=False)
season = pd.read_csv(fileList[3], verbose=False)
teams = pd.read_csv(fileList[4], verbose=False)
tournComp = pd.read_csv(fileList[5], verbose=False)
tournDet = pd.read_csv(fileList[6], verbose=False)
tournSeed = pd.read_csv(fileList[7], verbose=False)
tournSlot = pd.read_csv(fileList[8], verbose=False)

# Create Win/Loss data

In [4]:
regSeasonComp.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [5]:
regSeasonComp['count']=1

In [6]:
regSeasonComp.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,count
0,1985,20,1228,81,1328,64,N,0,1
1,1985,25,1106,77,1354,70,H,0,1
2,1985,25,1112,63,1223,56,H,0,1
3,1985,25,1165,70,1432,54,H,0,1
4,1985,25,1192,86,1447,74,H,0,1


In [7]:
#WinCount=regSeasonComp.groupby(by=["Season","Wteam"]).size()


In [8]:
#regSeasonComp.groupby(by=["Season","Wteam"])['Season','Wteam'].count()

In [9]:
wins = regSeasonComp["count"].groupby([regSeasonComp["Season"],regSeasonComp["Wteam"]]).sum().to_frame()
wins.reset_index(inplace=True)
wins.head()

,Season,Wteam,count
0,1985,1102,5
1,1985,1103,9
2,1985,1104,21
3,1985,1106,10
4,1985,1108,19


In [10]:
loss = regSeasonComp["count"].groupby([regSeasonComp["Season"],regSeasonComp["Lteam"]]).sum().to_frame()
loss.reset_index(inplace=True)
loss.head()

,Season,Lteam,count
0,1985,1102,19
1,1985,1103,14
2,1985,1104,9
3,1985,1106,14
4,1985,1108,6


In [11]:
winlocrec = regSeasonComp["count"].groupby(
    [regSeasonComp["Season"],regSeasonComp["Wteam"],regSeasonComp["Wloc"]]).sum().to_frame()
winlocrec=winlocrec.unstack(level=-1)
winlocrec.reset_index(inplace=True)
winlocrec.columns = ['Season','Team','Awins','Hwins','Nwins']
winlocrec["Nwins"].fillna(value=0,inplace= True)
print(winlocrec.head())

loslocrec = regSeasonComp["count"].groupby(
    [regSeasonComp["Season"],regSeasonComp["Lteam"],regSeasonComp["Wloc"]]).sum().to_frame()
loslocrec=loslocrec.unstack(level=-1)
loslocrec.reset_index(inplace=True)
loslocrec.columns = ['Season','Team','Hloss','Aloss','Nloss']
loslocrec["Nloss"].fillna(value=0,inplace= True)
print(loslocrec.head())

winlossloc = winlocrec.merge(loslocrec, left_on=['Season','Team'], right_on=['Season','Team'],how='outer' )

winlossloc.head()



   Season  Team  Awins  Hwins  Nwins
0    1985  1102    3.0    2.0    0.0
1    1985  1103    2.0    7.0    0.0
2    1985  1104    5.0   15.0    1.0
3    1985  1106    1.0    8.0    1.0
4    1985  1108    8.0    8.0    3.0
   Season  Team  Hloss  Aloss  Nloss
0    1985  1102    8.0    9.0    2.0
1    1985  1103    6.0    8.0    0.0
2    1985  1104    1.0    6.0    2.0
3    1985  1106    1.0   10.0    3.0
4    1985  1108    1.0    4.0    1.0


,Season,Team,Awins,Hwins,Nwins,Hloss,Aloss,Nloss
0,1985,1102,3.0,2.0,0.0,8.0,9.0,2.0
1,1985,1103,2.0,7.0,0.0,6.0,8.0,0.0
2,1985,1104,5.0,15.0,1.0,1.0,6.0,2.0
3,1985,1106,1.0,8.0,1.0,1.0,10.0,3.0
4,1985,1108,8.0,8.0,3.0,1.0,4.0,1.0


In [12]:
#testinner = wins.merge(loss, left_on=['Season','Wteam'], right_on=['Season','Lteam'],how='inner' )
testouter = wins.merge(loss, left_on=['Season','Wteam'], right_on=['Season','Lteam'],how='outer' )

In [13]:
#print(testinner.head())
testouter.head()

,Season,Wteam,count_x,Lteam,count_y
0,1985,1102.0,5.0,1102.0,19.0
1,1985,1103.0,9.0,1103.0,14.0
2,1985,1104.0,21.0,1104.0,9.0
3,1985,1106.0,10.0,1106.0,14.0
4,1985,1108.0,19.0,1108.0,6.0


In [14]:
testouter["count_x"].fillna(value=0,inplace= True)
testouter["count_y"].fillna(value=0,inplace= True)
testouter["Wteam"].fillna(value=testouter["Lteam"],inplace= True)
testouter.ix[testouter["count_x"].idxmin()]

Season     1986.0
Wteam      1159.0
count_x       0.0
Lteam      1159.0
count_y      22.0
Name: 10172, dtype: float64

In [15]:
testouter.drop('Lteam',axis=1,inplace=True)
testouter.columns = ['Season','Team','Wins','Losses']

In [16]:
testouter['Team']=testouter['Team'].astype(int)
testouter['Wins']=testouter['Wins'].astype(int)
testouter['Losses']=testouter['Losses'].astype(int)
testouter.head()

,Season,Team,Wins,Losses
0,1985,1102,5,19
1,1985,1103,9,14
2,1985,1104,21,9
3,1985,1106,10,14
4,1985,1108,19,6


In [17]:
W_L = testouter.merge(winlossloc, left_on=['Season','Team'], right_on=['Season','Team'],how='outer' )
W_L.head()

,Season,Team,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss
0,1985,1102,5,19,3.0,2.0,0.0,8.0,9.0,2.0
1,1985,1103,9,14,2.0,7.0,0.0,6.0,8.0,0.0
2,1985,1104,21,9,5.0,15.0,1.0,1.0,6.0,2.0
3,1985,1106,10,14,1.0,8.0,1.0,1.0,10.0,3.0
4,1985,1108,19,6,8.0,8.0,3.0,1.0,4.0,1.0


# Create Points for/against

In [18]:
regSeasonComp.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,count
0,1985,20,1228,81,1328,64,N,0,1
1,1985,25,1106,77,1354,70,H,0,1
2,1985,25,1112,63,1223,56,H,0,1
3,1985,25,1165,70,1432,54,H,0,1
4,1985,25,1192,86,1447,74,H,0,1


In [19]:
winTeamScore = regSeasonComp["Wscore"].groupby([regSeasonComp["Season"],regSeasonComp["Wteam"]]).sum().to_frame()
winTeamScore.reset_index(inplace=True)
#print('1.',len(winTeamScore))
#winTeamScore.dropna(inplace=True)
#print('1.',len(winTeamScore))

lossTeamScore = regSeasonComp["Lscore"].groupby([regSeasonComp["Season"],regSeasonComp["Lteam"]]).sum().to_frame()
lossTeamScore.reset_index(inplace=True)
#print('2.',len(lossTeamScore))
#lossTeamScore.dropna(inplace=True)
#print('2.',len(lossTeamScore))

winOtherTeamScore = regSeasonComp["Lscore"].groupby([regSeasonComp["Season"],regSeasonComp["Wteam"]]).sum().to_frame()
lossOtherTeamScore = regSeasonComp["Wscore"].groupby([regSeasonComp["Season"],regSeasonComp["Lteam"]]).sum().to_frame()
winOtherTeamScore.reset_index(inplace=True)
lossOtherTeamScore.reset_index(inplace=True)
#print('3.',len(winOtherTeamScore))
#lossOtherTeamScore.dropna(inplace=True)
#print('3.',len(winOtherTeamScore))
#print('4.',len(lossOtherTeamScore))
#winOtherTeamScore.dropna(inplace=True)
#print('4.',len(lossOtherTeamScore))

#print(winOtherTeamScore.head(),lossOtherTeamScore.head())

OtherTeamScore = winOtherTeamScore.merge(lossOtherTeamScore, left_on=['Season','Wteam'], right_on=['Season','Lteam'],how='outer')
#print('5.',len(OtherTeamScore))
OtherTeamScore['Wscore'].fillna(0,inplace=True)
OtherTeamScore['Lscore'].fillna(0,inplace=True)
OtherTeamScore['pointsAgainst'] = OtherTeamScore.apply(lambda x: x['Wscore']+ x['Lscore'], axis=1)
#print('5.',len(OtherTeamScore))
#print('5.',len(OtherTeamScore))
OtherTeamScore['Team']=OtherTeamScore['Wteam']
OtherTeamScore['Team'].fillna(OtherTeamScore['Lteam'],inplace=True)
OtherTeamScore.drop(['Wteam','Lscore','Lteam','Wscore'],axis=1,inplace=True)
#print(OtherTeamScore.head())

TeamScore = winTeamScore.merge(lossTeamScore, left_on=['Season','Wteam'], right_on=['Season','Lteam'],how='outer' )
TeamScore['Wscore'].fillna(0,inplace=True)
TeamScore['Lscore'].fillna(0,inplace=True)
TeamScore['pointsFor'] = TeamScore.apply(lambda x: x['Wscore']+ x['Lscore'], axis=1)
TeamScore['Team']=TeamScore['Wteam']
TeamScore['Team'].fillna(TeamScore['Lteam'],inplace=True)
TeamScore.drop(['Wteam','Wscore','Lteam','Lscore'],axis=1,inplace=True)
#print(TeamScore.head())
#print('6.',len(TeamScore))

Points = TeamScore.merge(OtherTeamScore, left_on=['Season','Team'], right_on=['Season','Team'],how='outer' )
#print('7.',len(Points))
#print(Points.head())

#Points.drop(['Wscore_x','Lteam_x','Lscore_x','Lscore_y','Lteam_y','Wscore_y'],axis=1,inplace=True)
#Points.columns = ['Season','Team','pointsFor','pointsAgainst']

#Points.dropna(inplace=True)
Points['Team']=Points['Team'].astype(int)
Points['pointsFor']=Points['pointsFor'].astype(int)
Points['pointsAgainst']=Points['pointsAgainst'].astype(int)

#print('8.',len(Points))

Points.head()


,Season,pointsFor,Team,pointsAgainst
0,1985,1514,1102,1653
1,1985,1404,1103,1474
2,1985,2055,1104,1821
3,1985,1719,1106,1810
4,1985,2075,1108,1876


In [20]:
len(Points)

10186

In [21]:
print(len(W_L))
W_L['Awins'].fillna(0,inplace=True)
W_L['Hwins'].fillna(0,inplace=True)
W_L['Nwins'].fillna(0,inplace=True)
W_L['Hloss'].fillna(0,inplace=True)
W_L['Aloss'].fillna(0,inplace=True)
W_L['Nloss'].fillna(0,inplace=True)
W_L['Awins']=W_L['Awins'].astype(int)
W_L['Hwins']=W_L['Hwins'].astype(int)
W_L['Nwins']=W_L['Nwins'].astype(int)
W_L['Hloss']=W_L['Hloss'].astype(int)
W_L['Aloss']=W_L['Aloss'].astype(int)
W_L['Nloss']=W_L['Nloss'].astype(int)
W_L.head()

10186


,Season,Team,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss
0,1985,1102,5,19,3,2,0,8,9,2
1,1985,1103,9,14,2,7,0,6,8,0
2,1985,1104,21,9,5,15,1,1,6,2
3,1985,1106,10,14,1,8,1,1,10,3
4,1985,1108,19,6,8,8,3,1,4,1


In [22]:
TeamOverview = Points.merge(W_L, left_on=['Season','Team'], right_on=['Season','Team'],how='outer' )
print(len(TeamOverview))
TeamOverview.head()

10186


,Season,pointsFor,Team,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss
0,1985,1514,1102,1653,5,19,3,2,0,8,9,2
1,1985,1404,1103,1474,9,14,2,7,0,6,8,0
2,1985,2055,1104,1821,21,9,5,15,1,1,6,2
3,1985,1719,1106,1810,10,14,1,8,1,1,10,3
4,1985,2075,1108,1876,19,6,8,8,3,1,4,1


# Add team seading

In [23]:
#print(tournComp.head())
#print(tournDet.head())
print(tournSeed.head())
#print(tournSlot.head())

   Season Seed  Team
0    1985  W01  1207
1    1985  W02  1210
2    1985  W03  1228
3    1985  W04  1260
4    1985  W05  1374


In [24]:
print(len(TeamOverview))
TeamOverview = TeamOverview.merge(tournSeed,left_on=['Season','Team'], right_on=['Season','Team'],how='left')
print(len(TeamOverview))
TeamOverview.head()

10186
10186


,Season,pointsFor,Team,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss,Seed
0,1985,1514,1102,1653,5,19,3,2,0,8,9,2,NaN
1,1985,1404,1103,1474,9,14,2,7,0,6,8,0,NaN
2,1985,2055,1104,1821,21,9,5,15,1,1,6,2,X07
3,1985,1719,1106,1810,10,14,1,8,1,1,10,3,NaN
4,1985,2075,1108,1876,19,6,8,8,3,1,4,1,NaN


In [25]:
#TeamOverview['Seed'].fillna('A',inplace=True)
TeamOverview['SeedNum']=TeamOverview['Seed'].apply(
    lambda x: int(x.lstrip('ABCDEFGHIJKLMNOPQRSTUVWXYZ').rstrip('abcdefghijklmnopqrstuvwxyz')) if type(x)==str else x)
#TeamOverview['Seed'].replace(to_replace='A',value=np.nan(), inplace=True)
#TeamOverview['Seed'].fillna('A',inplace=True)
print(type(TeamOverview['SeedNum'][2]))

TeamOverview.head()

<class 'numpy.float64'>


,Season,pointsFor,Team,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss,Seed,SeedNum
0,1985,1514,1102,1653,5,19,3,2,0,8,9,2,NaN,NaN
1,1985,1404,1103,1474,9,14,2,7,0,6,8,0,NaN,NaN
2,1985,2055,1104,1821,21,9,5,15,1,1,6,2,X07,7.0
3,1985,1719,1106,1810,10,14,1,8,1,1,10,3,NaN,NaN
4,1985,2075,1108,1876,19,6,8,8,3,1,4,1,NaN,NaN


# Best win
to start this will be by best record team. Tie breaker will be turny rank

In [26]:
regSeasonComp.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,count
0,1985,20,1228,81,1328,64,N,0,1
1,1985,25,1106,77,1354,70,H,0,1
2,1985,25,1112,63,1223,56,H,0,1
3,1985,25,1165,70,1432,54,H,0,1
4,1985,25,1192,86,1447,74,H,0,1


In [27]:
games = regSeasonComp[['Season','Wteam','Lteam']]
games = games.merge(
    TeamOverview[['Season','Team','Wins','Losses','SeedNum']],left_on=['Season','Lteam'], right_on=['Season','Team'],how='left')
games['winPercent'] = games.apply(lambda x: x['Wins']/(x['Wins']+ x['Losses']), axis=1)
games.head()

,Season,Wteam,Lteam,Team,Wins,Losses,SeedNum,winPercent
0,1985,1228,1328,1328,25,5,1.0,0.833333
1,1985,1106,1354,1354,9,15,NaN,0.375000
2,1985,1112,1223,1223,17,8,NaN,0.680000
3,1985,1165,1432,1432,11,12,NaN,0.478261
4,1985,1192,1447,1447,8,16,NaN,0.333333


In [28]:
games.sort_values(['Season','Wteam','winPercent','SeedNum'], ascending=[1,1,0,1], inplace=True)
games.head()

,Season,Wteam,Lteam,Team,Wins,Losses,SeedNum,winPercent
3563,1985,1102,1140,1140,15,14,NaN,0.517241
3096,1985,1102,1461,1461,12,14,NaN,0.461538
3157,1985,1102,1218,1218,10,18,NaN,0.357143
137,1985,1102,1225,1225,5,21,NaN,0.192308
407,1985,1102,1109,1109,1,23,NaN,0.041667


'''
teamorder =games[['Team','Wteam','Season']]
teamorder.reset_index(inplace=True)
teamorder.reset_index(inplace=True)
teamorder.drop('index',axis=1,inplace=True)
teamorder.columns = ['WinRank','Team','Wteam','Season']
teamorder.head()
'''

'''games = games.merge(
    teamorder,left_on=['Season','Wteam','Team'], right_on=['Season','Wteam','Team'],how='left')
games.head()'''

In [29]:
games.reset_index(inplace=True)
games.reset_index(inplace=True)
games.drop('index',axis=1,inplace=True)
games.columns = ['BestWin','Season','Wteam','Lteam','Team','Wins','Losses','SeedNum','winPercent']
games.head()

,BestWin,Season,Wteam,Lteam,Team,Wins,Losses,SeedNum,winPercent
0,0,1985,1102,1140,1140,15,14,NaN,0.517241
1,1,1985,1102,1461,1461,12,14,NaN,0.461538
2,2,1985,1102,1218,1218,10,18,NaN,0.357143
3,3,1985,1102,1225,1225,5,21,NaN,0.192308
4,4,1985,1102,1109,1109,1,23,NaN,0.041667


In [30]:
idx = games.groupby(['Season','Wteam'])['BestWin'].transform(min) == games['BestWin']
bestwin = games[idx]
bestwin.drop(['BestWin','Lteam','Wins','Losses'],axis=1,inplace=True)
bestwin.columns = ['Season','Wteam','BestWin','BestWinSeed','BestWinWin%']
bestwin.head()

C:\Anaconda2\envs\py35\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Season,Wteam,BestWin,BestWinSeed,BestWinWin%
0,1985,1102,1140,NaN,0.517241
5,1985,1103,1293,NaN,0.666667
14,1985,1104,1208,6.0,0.703704
35,1985,1106,1380,16.0,0.642857
45,1985,1108,1380,16.0,0.642857


In [31]:
TeamOverview = TeamOverview.merge(bestwin,left_on=['Season','Team'], right_on=['Season','Wteam'],how='left')
TeamOverview.drop(['Wteam'],axis=1,inplace=True)
TeamOverview.head()

,Season,pointsFor,Team,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss,Seed,SeedNum,BestWin,BestWinSeed,BestWinWin%
0,1985,1514,1102,1653,5,19,3,2,0,8,9,2,NaN,NaN,1140.0,NaN,0.517241
1,1985,1404,1103,1474,9,14,2,7,0,6,8,0,NaN,NaN,1293.0,NaN,0.666667
2,1985,2055,1104,1821,21,9,5,15,1,1,6,2,X07,7.0,1208.0,6.0,0.703704
3,1985,1719,1106,1810,10,14,1,8,1,1,10,3,NaN,NaN,1380.0,16.0,0.642857
4,1985,2075,1108,1876,19,6,8,8,3,1,4,1,NaN,NaN,1380.0,16.0,0.642857


In [32]:
TeamOverview.head()

,Season,pointsFor,Team,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss,Seed,SeedNum,BestWin,BestWinSeed,BestWinWin%
0,1985,1514,1102,1653,5,19,3,2,0,8,9,2,NaN,NaN,1140.0,NaN,0.517241
1,1985,1404,1103,1474,9,14,2,7,0,6,8,0,NaN,NaN,1293.0,NaN,0.666667
2,1985,2055,1104,1821,21,9,5,15,1,1,6,2,X07,7.0,1208.0,6.0,0.703704
3,1985,1719,1106,1810,10,14,1,8,1,1,10,3,NaN,NaN,1380.0,16.0,0.642857
4,1985,2075,1108,1876,19,6,8,8,3,1,4,1,NaN,NaN,1380.0,16.0,0.642857


In [33]:
TeamOverview = TeamOverview[['Season','Team','pointsFor','pointsAgainst',
                             'Wins','Losses','Awins','Hwins','Nwins','Hloss',
                            'Aloss','Nloss','Seed','SeedNum','BestWin','BestWinSeed','BestWinWin%']]
TeamOverview.head()

,Season,Team,pointsFor,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss,Seed,SeedNum,BestWin,BestWinSeed,BestWinWin%
0,1985,1102,1514,1653,5,19,3,2,0,8,9,2,NaN,NaN,1140.0,NaN,0.517241
1,1985,1103,1404,1474,9,14,2,7,0,6,8,0,NaN,NaN,1293.0,NaN,0.666667
2,1985,1104,2055,1821,21,9,5,15,1,1,6,2,X07,7.0,1208.0,6.0,0.703704
3,1985,1106,1719,1810,10,14,1,8,1,1,10,3,NaN,NaN,1380.0,16.0,0.642857
4,1985,1108,2075,1876,19,6,8,8,3,1,4,1,NaN,NaN,1380.0,16.0,0.642857


In [34]:
TeamOverview.to_csv(path_or_buf="../TeamOverview.csv",index=False)

In [35]:
sampleSub.head()

,id,pred
0,2013_1103_1107,0.5
1,2013_1103_1112,0.5
2,2013_1103_1125,0.5
3,2013_1103_1129,0.5
4,2013_1103_1137,0.5


## Create addtional Metrics for Models 6-?

In [105]:
TeamOverview2 = TeamOverview.copy()
TeamOverview2.head()

,Season,Team,pointsFor,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,Aloss,Nloss,Seed,SeedNum,BestWin,BestWinSeed,BestWinWin%
0,1985,1102,1514,1653,5,19,3,2,0,8,9,2,NaN,NaN,1140.0,NaN,0.517241
1,1985,1103,1404,1474,9,14,2,7,0,6,8,0,NaN,NaN,1293.0,NaN,0.666667
2,1985,1104,2055,1821,21,9,5,15,1,1,6,2,X07,7.0,1208.0,6.0,0.703704
3,1985,1106,1719,1810,10,14,1,8,1,1,10,3,NaN,NaN,1380.0,16.0,0.642857
4,1985,1108,2075,1876,19,6,8,8,3,1,4,1,NaN,NaN,1380.0,16.0,0.642857


#### First set will be points for and against averages for each location

In [106]:
regSeasonComp.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,count
0,1985,20,1228,81,1328,64,N,0,1
1,1985,25,1106,77,1354,70,H,0,1
2,1985,25,1112,63,1223,56,H,0,1
3,1985,25,1165,70,1432,54,H,0,1
4,1985,25,1192,86,1447,74,H,0,1


In [107]:

pointavgL = regSeasonComp["Lscore"].groupby([regSeasonComp["Season"],regSeasonComp["Lteam"],regSeasonComp["Wloc"]]).mean().to_frame()
pointavgL = pointavgL.round( decimals=3)
pointavgL = pointavgL.unstack(level=-1)
pointavgL.reset_index(inplace=True)
pointavgL.columns = ['Season','Team','HlossAvg','AlossAvg','NlossAvg']
pointavgL['NlossAvg'].fillna(value=pointavgL['AlossAvg'], inplace=True)
#print(pointavgL.head())
pointavgW = regSeasonComp["Wscore"].groupby([regSeasonComp["Season"],regSeasonComp["Wteam"],regSeasonComp["Wloc"]]).mean().to_frame()
pointavgW = pointavgW.round( decimals=3)
pointavgW = pointavgW.unstack(level=-1)
pointavgW.reset_index(inplace=True)
pointavgW.columns = ['Season','Team','AwinAvg','HwinAvg','NwinAvg']
pointavgW['NwinAvg'].fillna(value=pointavgW['AwinAvg'], inplace=True)
#pointavgW.head()
pointavg = pointavgL.merge(pointavgW, left_on=['Season','Team'], right_on=['Season','Team'],how='outer' )
pointavg.head()

,Season,Team,HlossAvg,AlossAvg,NlossAvg,AwinAvg,HwinAvg,NwinAvg
0,1985,1102,63.250,60.222,55.500,70.667,71.500,70.667
1,1985,1103,53.833,56.125,56.125,79.500,67.571,79.500
2,1985,1104,67.000,61.500,52.500,69.600,74.133,54.000
3,1985,1106,77.000,72.300,56.000,72.000,73.625,90.000
4,1985,1108,72.000,75.500,70.000,80.125,93.625,80.333


In [108]:
TeamOverview2 = TeamOverview2.merge(pointavg, left_on=['Season','Team'], right_on=['Season','Team'],how='left' )
TeamOverview2.head()


,Season,Team,pointsFor,pointsAgainst,Wins,Losses,Awins,Hwins,Nwins,Hloss,...,SeedNum,BestWin,BestWinSeed,BestWinWin%,HlossAvg,AlossAvg,NlossAvg,AwinAvg,HwinAvg,NwinAvg
0,1985,1102,1514,1653,5,19,3,2,0,8,...,NaN,1140.0,NaN,0.517241,63.250,60.222,55.500,70.667,71.500,70.667
1,1985,1103,1404,1474,9,14,2,7,0,6,...,NaN,1293.0,NaN,0.666667,53.833,56.125,56.125,79.500,67.571,79.500
2,1985,1104,2055,1821,21,9,5,15,1,1,...,7.0,1208.0,6.0,0.703704,67.000,61.500,52.500,69.600,74.133,54.000
3,1985,1106,1719,1810,10,14,1,8,1,1,...,NaN,1380.0,16.0,0.642857,77.000,72.300,56.000,72.000,73.625,90.000
4,1985,1108,2075,1876,19,6,8,8,3,1,...,NaN,1380.0,16.0,0.642857,72.000,75.500,70.000,80.125,93.625,80.333


In [109]:
TeamOverview2.to_csv(path_or_buf="../TeamOverview2.csv",index=False)

In [115]:
TeamOverview2['NwinAvg'].min()

38.0